In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Badwater 267 VR 2025/gpx.csv', low_memory=False)
df.head(1)

,lat,lon,ele,strava_id
0,33.88152,-117.760172,131.6,14113701207


In [12]:
meta_df = pd.read_csv('/content/drive/MyDrive/Badwater 267 VR 2025/runsignup.csv', low_memory=False)
meta_df.head(1)

,place,bib,name,city,state,country,clock time,distance in miles,progress,elevation gain,pace,date,miles,time,elevation,strava_link,strava_id
0,1,118,Noah White,Slingerlands,NY,US,34:37:36,267.0,100%,"6,541ft (1,993.7m)",7:47,20.04.2025,1.3,8:25,0,https://www.strava.com/activities/14236433263,14236433263


In [15]:
merged_df = df.merge(meta_df[['name', 'strava_id']], on='strava_id', how='inner')
merged_df.head(1)

,lat,lon,ele,strava_id,name
0,33.88152,-117.760172,131.6,14113701207,John Goss


In [23]:
counts = merged_df.groupby(['strava_id', 'name']).size().reset_index(name='point_count')

Q1 = counts['point_count'].quantile(0.25)
Q3 = counts['point_count'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

counts['is_outlier'] = (counts['point_count'] < lower_bound) | (counts['point_count'] > upper_bound)
counts_sorted = counts.sort_values(by='point_count', ascending=False)

counts_sorted[counts_sorted['is_outlier'] == True].head(5)

,strava_id,name,point_count,is_outlier
458,14165012787,Kevin Dockemeyer,154791,True
1014,LylsYct9CSb,Kevin Koncilja,78702,True
74,14094056631,Angela Hawley,73300,True
1011,LoZMLcs4wSb,Jeffrey Ruiz,59498,True
143,14106357641,Brian Watson,47107,True
